In [ ]:
#pip install plotly_express

In [ ]:
#pip install --upgrade plotly


# Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from itertools import cycle

pd.set_option('max_columns', 50)
plt.style.use('bmh')

color_pal = plt.rcParams['axes.prop_cycle'].by_key()['color']
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

# ------------------
import os
import gc
import time
import math
import datetime
from math import log, floor
from sklearn.neighbors import KDTree


from pathlib import Path
from sklearn.utils import shuffle
from tqdm.notebook import tqdm as tqdm

from matplotlib import colors
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import pywt
from statsmodels.robust import mad

#-----------------
import warnings
warnings.filterwarnings('ignore')

from tqdm.notebook import tqdm
import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, Input, Embedding, Dropout, concatenate, Flatten
from tensorflow.keras.models import Model

In [ ]:
# from google.colab import drive
# drive.mount("/content/drive")

# Read the Data

The data, covers stores in three US States (California, Texas, and Wisconsin) and includes item level, department, product categories, and store details.


    calendar.csv - Contains information about the dates on which the products are sold.
    sales_train_validation.csv - Contains the historical daily unit sales data per product and store [d_1 - d_1913]
    sample_submission.csv - The correct format for submissions. Reference the Evaluation tab for more info.
    sell_prices.csv - Contains information about the price of the products sold per store and date.
    sales_train_evaluation.csv - Available one month before competition deadline. Will include sales [d_1 - d_1941]


The dataset involves the unit sales of 3,049 products, classified in 3 product categories (Hobbies, Foods, and Household) and 7 product departments, in which the categories are disaggregated.  The products are sold across ten stores, located in three States (CA, TX, and WI).
<img src='https://i.imgur.com/Ae5QBi9.png' style="width:500px;height:300px;">

In [ ]:
%%time
calendar = pd.read_csv("../input/m5-forecasting-accuracy/calendar.csv")
sales_train_val = pd.read_csv("../input/m5-forecasting-accuracy/sales_train_validation.csv")
sample_submission = pd.read_csv("../input/m5-forecasting-accuracy/sample_submission.csv")
selling_prices = pd.read_csv("../input/m5-forecasting-accuracy/sell_prices.csv")

In [ ]:
print(sales_train_val.shape)
sales_train_val.head()

In [ ]:
print(calendar.shape)
calendar.head()

In [ ]:
print(selling_prices.shape)
selling_prices

# EDA

## Distribution of Items

Let's find the total count of items of each of the product over the total time period.

In [ ]:
group = sales_train_val.groupby(['state_id','store_id','cat_id','dept_id'],as_index=False)['item_id'].count().dropna()
group['USA'] = 'United States of America' # Assigning parent
group.rename(columns={'state_id':'State','store_id':'Store','cat_id':'Category','dept_id':'Department','item_id':'Count'},inplace=True)

# Creates a Treemap for all the products
fig = px.treemap(group, path=['USA', 'State', 'Store', 'Category', 'Department'], 
                  values='Count',
                  color='Count',
                  color_continuous_scale= px.colors.sequential.Sunset,
                  title='Walmart: Distribution of Items')

fig.update_layout(template='seaborn')
fig.show()

## Visualize data for single item

### Visualize for days

In [ ]:
req_col = []

# Append all the days in req_col array
for c in sales_train_val.columns:
  if 'd_' in c:
    req_col.append(c)
    
#print(req_col)

Let's visualize sales for FOODS_3_090_CA_3 across the time period

1. Sample Item = "FOODS_3_090_CA_3"

In [ ]:
sample_item1 = sales_train_val.loc[sales_train_val['id']=="FOODS_3_090_CA_3_validation"]
sample_item1 = sample_item1.set_index('id')[req_col]
sample_item1 = sample_item1.T
sample_item1.plot(figsize = (15,5), title = 'FOODS_3_090_CA_3_validation sales by d number', color = next(color_cycle))
plt.show()

### Visualize the dates

In the above figure, the data is visualized for days. Let's covert the X axis to Dates format. It can be done by merging sales data with Calendar data.

In [ ]:
# Selecting days column
sample_item1 = sales_train_val.loc[sales_train_val['id']=='FOODS_3_090_CA_3_validation'][req_col].T
# Naming it by correct name
sample_item1 = sample_item1.rename(columns = {8412:'FOODS_3_090_CA_3'})
# Making the index as days i.e. d
sample_item1 = sample_item1.reset_index().rename(columns = {'index':'d'})
sample_item1 = sample_item1.merge(calendar,how = 'left', validate="1:1")

In [ ]:
print(sample_item1.shape)
sample_item1.head()

In [ ]:
sample_item1.set_index('date')["FOODS_3_090_CA_3"].plot(figsize = (15,5), title = 'FOODS_3_090_CA_3 sales by date values',\
                                                        color = next(color_cycle)) 
plt.show()

Let's visualize for another items and see the variation.

2. Sample Item = HOBBIES_1_234_CA_3

In [ ]:
sample_item2 = sales_train_val.loc[sales_train_val['id']=='HOBBIES_1_234_CA_3_validation'][req_col].T
sample_item2 = sample_item2.rename(columns = {6324:'HOBBIES_1_234_CA_3'})
sample_item2 = sample_item2.reset_index().rename(columns = {'index':'d'})
sample_item2 = sample_item2.merge(calendar,how = 'left', validate="1:1")
sample_item2.set_index('date')["HOBBIES_1_234_CA_3"].plot(figsize = (15,5), \
                              title = 'HOBBIES_1_234_CA_3 sales by date values', color = next(color_cycle)) 
plt.show()

3. Sample Item = HOUSEHOLD_1_118_CA_3

In [ ]:
sample_item3 = sales_train_val.loc[sales_train_val['id']=='HOUSEHOLD_1_118_CA_3_validation'][req_col].T
sample_item3 = sample_item3.rename(columns = {6776:'HOUSEHOLD_1_118_CA_3'})
sample_item3 = sample_item3.reset_index().rename(columns = {'index':'d'})
sample_item3 = sample_item3.merge(calendar,how = 'left', validate="1:1")
sample_item3.set_index('date')["HOUSEHOLD_1_118_CA_3"].plot(figsize = (15,5), \
                              title = 'HOUSEHOLD_1_118_CA_3 sales by date values',  color = next(color_cycle)) 
plt.show()

In [ ]:
print(sample_item3.shape)
sample_item3.head()

### Visualize for day of week, Month and Year

Considering the above three items, we will analyze the flow of data according to -
1. Day of the week
2. Month
3. Year

In [ ]:
sample_items = ['FOODS_3_090_CA_3', 'HOBBIES_1_234_CA_3', 'HOUSEHOLD_1_118_CA_3']
items = [sample_item1, sample_item2, sample_item3]

for i in [0,1,2]:
    fig, (ax1,ax2,ax3) = plt.subplots(1,3,figsize=(15,3))
    items[i].groupby('wday').mean()[sample_items[i]].plot(kind='line',title='average sale: day of week', lw = 5, ax=ax1, color = color_pal[0])
    items[i].groupby('month').mean()[sample_items[i]].plot(kind='line',title='average sale: month', lw = 5, ax=ax2, color = color_pal[4])
    items[i].groupby('year').mean()[sample_items[i]].plot(kind='line',title='average sale: year', lw = 5, ax=ax3,  color = color_pal[2])
    fig.suptitle('Trends for item:{}'.format(sample_items[i]),y=1.1)
    plt.tight_layout()
    plt.show()
    

### Visualize for 20 sales items

Let's visualize the data for random 20 items

In [ ]:
twenty_examples = sales_train_val.sample(20, random_state=529)
twenty_examples = twenty_examples.set_index('id')[req_col].T
twenty_examples = twenty_examples.merge(calendar.set_index('d')['date'],left_index=True,right_index=True,validate='1:1')
twenty_examples = twenty_examples.set_index('date')

In [ ]:
fig, axs = plt.subplots(10, 2, figsize=(15, 20))

axs = axs.flatten()
ax_idx = 0

for item in twenty_examples.columns:
    twenty_examples[item].plot(title=item, ax=axs[ax_idx], color = next(color_cycle))
    ax_idx += 1
plt.tight_layout()
plt.show()

It is common to see an item unavailable for a period of time.

Some items only sell 1 or less in a day, making it very hard to predict.

## Sales over time by Type

In [ ]:
# How many types of Products are there?
sales_train_val['cat_id'].unique()

In [ ]:
count_by_type = sales_train_val.groupby('cat_id').count()['id']

In [ ]:
count_by_type

In [ ]:
count_by_type.plot(figsize = (15,5), title = 'Count of Items by Category')
plt.show()

In [ ]:
count_by_type.plot(kind = 'barh',figsize = (15,5), title = 'Count of Items by Category')
plt.show()

In [ ]:
past_sales = sales_train_val.set_index('id')[req_col] \
    .T \
    .merge(calendar.set_index('d')['date'],
           left_index=True,
           right_index=True,
            validate='1:1') \
    .set_index('date')

In [ ]:
print(past_sales.shape)
past_sales.head()

In [ ]:
for i in sales_train_val['cat_id'].unique():
    items_col = [c for c in past_sales.columns if i in c]
    past_sales[items_col] \
        .sum(axis=1) \
        .plot(figsize=(15, 5),
              alpha=0.8,
              title='Total Sales by Item Type')
plt.legend(sales_train_val['cat_id'].unique())
plt.show()



In [ ]:
past_sales_clipped = past_sales.clip(0, 1)

for i in sales_train_val['cat_id'].unique():
    items_col = [c for c in past_sales.columns if i in c]
    (past_sales_clipped[items_col] \
        .mean(axis=1) * 100) \
        .plot(figsize=(15, 5),
              alpha=0.8,
              title='Inventory Sale Percentage by Date',
              style='.')
plt.ylabel('% of Inventory with at least 1 sale')
plt.legend(sales_train_val['cat_id'].unique())
plt.show()


It is observed that highest sales is observed for FOODS and that too in increasing manner.

## Sales by Store

In [ ]:
store_list = selling_prices['store_id'].unique()

for s in store_list:
    store_items = [c for c in past_sales.columns if s in c]
    past_sales[store_items] \
        .sum(axis=1) \
        .rolling(90).mean() \
        .plot(figsize=(15, 5),
              alpha=0.8,
              title='Rolling 90 Day Average Total Sales (10 stores)')
plt.legend(store_list)
plt.show()

In [ ]:
fig, axes = plt.subplots(5, 2, figsize=(15, 10), sharex=True)
axes = axes.flatten()
ax_idx = 0
for s in store_list:
    store_items = [c for c in past_sales.columns if s in c]
    past_sales[store_items] \
        .sum(axis=1) \
        .rolling(7).mean() \
        .plot(alpha=1,
              ax=axes[ax_idx],
              title=s,
              lw=3,
              color=next(color_cycle))
    ax_idx += 1
# plt.legend(store_list)
plt.suptitle('Weekly Sale Trends by Store ID')
plt.tight_layout()
plt.show()

## Sales Price

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
stores = []

for store, d in selling_prices.query('item_id == "FOODS_3_090"').groupby('store_id'):
    d.plot(x='wm_yr_wk',
          y='sell_price',
          style='.',
          color=next(color_cycle),
          figsize=(15, 5),
          title='FOODS_3_090 sale price over time',
         ax=ax,
          legend=store)
    stores.append(store)
    plt.legend()
    
plt.legend(stores)
plt.show()

In [ ]:
selling_prices['Category'] = selling_prices['item_id'].str.split('_', expand=True)[0]
fig, axs = plt.subplots(1, 3, figsize=(15, 4))
i = 0

for cat, d in selling_prices.groupby('Category'):
    ax = d['sell_price'] \
        .plot(kind='hist',
                         bins=20,
                         title=f'Distribution of {cat} prices',
                         ax=axs[i],
                                         color=next(color_cycle))
    ax.set_xlabel('Log(price)')
    i += 1
    
plt.tight_layout()

## Sample Sales

In [ ]:
ids = sorted(list(set(sales_train_val['id'])))
# d_cols = [c for c in sales_train_val.columns if 'd_' in c]
sales_train_val.loc[sales_train_val['id'] == ids[0]].set_index('id')[req_col]

Let's look for examples of 350 to 450 days

In [ ]:
x_1 = sales_train_val.loc[sales_train_val['id'] == ids[0]].set_index('id')[req_col].values[0][350:450]
x_2 = sales_train_val.loc[sales_train_val['id'] == ids[4]].set_index('id')[req_col].values[0][350:450]
x_3 = sales_train_val.loc[sales_train_val['id'] == ids[65]].set_index('id')[req_col].values[0][350:450]

In [ ]:
fig = make_subplots(rows=3, cols=1)

fig.add_trace(go.Scatter(x=np.arange(len(x_1)), y=x_1, showlegend=False,
                    mode='lines+markers', name="First sample",
                         marker=dict(color="mediumseagreen")),
             row=1, col=1)

fig.add_trace(go.Scatter(x=np.arange(len(x_2)), y=x_2, showlegend=False,
                    mode='lines+markers', name="Second sample",
                         marker=dict(color="violet")),
             row=2, col=1)

fig.add_trace(go.Scatter(x=np.arange(len(x_3)), y=x_3, showlegend=False,
                    mode='lines+markers', name="Third sample",
                         marker=dict(color="dodgerblue")),
             row=3, col=1)

fig.update_layout(height=1200, width=800, title_text="Sample sales snippets")
fig.show()

## Denoise the signal

### Normal Smoothing 

We will create a function to denoise our signal

In [ ]:
def maddest(d, axis=None):
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

def denoise_signal(x, wavelet='db4', level=1):
    coeff = pywt.wavedec(x, wavelet, mode="per")
    sigma = (1/0.6745) * maddest(coeff[-level])

    uthresh = sigma * np.sqrt(2*np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])

    return pywt.waverec(coeff, wavelet, mode='per')

In [ ]:
y_w1 = denoise_signal(x_1)
y_w2 = denoise_signal(x_2)
y_w3 = denoise_signal(x_3)


fig = make_subplots(rows=3, cols=1)

fig.add_trace(
    go.Scatter(x=np.arange(len(x_1)), mode='lines+markers', y=x_1, marker=dict(color="mediumaquamarine"), showlegend=False,
               name="Original signal"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(len(x_1)), y=y_w1, mode='lines', marker=dict(color="darkgreen"), showlegend=False,
               name="Denoised signal"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(len(x_2)), mode='lines+markers', y=x_2, marker=dict(color="thistle"), showlegend=False),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(len(x_2)), y=y_w2, mode='lines', marker=dict(color="purple"), showlegend=False),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(len(x_3)), mode='lines+markers', y=x_3, marker=dict(color="lightskyblue"), showlegend=False),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(len(x_3)), y=y_w3, mode='lines', marker=dict(color="navy"), showlegend=False),
    row=3, col=1
)

fig.update_layout(height=1200, width=800, title_text="Original (pale) vs. Denoised (dark) sales")
fig.show()

In the above graphs, the dark lineplots represent the denoised sales and the light lineplots represent the original sales. We can see that Wavelet denoising is able to successfully find the "general trend" in the sales data without getting distracted by the noise.

In [ ]:


fig, ax = plt.subplots(nrows=3, ncols=2, figsize=(30, 20))

ax[0, 0].plot(x_1, color='seagreen', marker='o') 
ax[0, 0].set_title('Original Sales', fontsize=24)
ax[0, 1].plot(y_w1, color='red', marker='.') 
ax[0, 1].set_title('After Wavelet Denoising', fontsize=24)

ax[1, 0].plot(x_2, color='seagreen', marker='o') 
ax[1, 0].set_title('Original Sales', fontsize=24)
ax[1, 1].plot(y_w2, color='red', marker='.') 
ax[1, 1].set_title('After Wavelet Denoising', fontsize=24)

ax[2, 0].plot(x_3, color='seagreen', marker='o') 
ax[2, 0].set_title('Original Sales', fontsize=24)
ax[2, 1].plot(y_w3, color='red', marker='.') 
ax[2, 1].set_title('After Wavelet Denoising', fontsize=24)

plt.show()

### Average Smoothing


Average smooting is a relatively simple way to denoise time series data. 

In this method, we take a "window" with a fixed size (like 10). We first place the window at the beginning of the time series (first ten elements) and calculate the mean of that section. We now move the window across the time series in the forward direction by a particular "stride", calculate the mean of the new window and repeat the process, until we reach the end of the time series. 

All the mean values we calculated are then concatenated into a new time series, which forms the denoised sales data.


In [ ]:
def average_smoothing(signal, kernel_size=3, stride=1):
    sample = []
    start = 0
    end = kernel_size
    
    while end <= len(signal):
        start = start + stride
        end = end + stride
        sample.extend(np.ones(end - start)*np.mean(signal[start:end]))
    
    return np.array(sample)

In [ ]:
%%time
y_a1 = average_smoothing(x_1)
y_a2 = average_smoothing(x_2)
y_a3 = average_smoothing(x_3)

fig = make_subplots(rows=3, cols=1)

fig.add_trace(
    go.Scatter(x=np.arange(len(x_1)), mode='lines+markers', y=x_1, marker=dict(color="lightskyblue"), showlegend=False,
               name="Original sales"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(len(x_1)), y=y_a1, mode='lines', marker=dict(color="navy"), showlegend=False,
               name="Denoised sales"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(len(x_2)), mode='lines+markers', y=x_2, marker=dict(color="thistle"), showlegend=False),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(len(x_2)), y=y_a2, mode='lines', marker=dict(color="indigo"), showlegend=False),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(len(x_3)), mode='lines+markers', y=x_3, marker=dict(color="mediumaquamarine"), showlegend=False),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(len(x_3)), y=y_a3, mode='lines', marker=dict(color="darkgreen"), showlegend=False),
    row=3, col=1
)

fig.update_layout(height=1200, width=800, title_text="Original (pale) vs. Denoised (dark) signals")
fig.show()

In [ ]:


fig, ax = plt.subplots(nrows=3, ncols=2, figsize=(30, 20))

ax[0, 0].plot(x_1, color='seagreen', marker='o') 
ax[0, 0].set_title('Original Sales', fontsize=24)
ax[0, 1].plot(y_a1, color='red', marker='.') 
ax[0, 1].set_title('After Wavelet Denoising', fontsize=24)

ax[1, 0].plot(x_2, color='seagreen', marker='o') 
ax[1, 0].set_title('Original Sales', fontsize=24)
ax[1, 1].plot(y_a2, color='red', marker='.') 
ax[1, 1].set_title('After Wavelet Denoising', fontsize=24)

ax[2, 0].plot(x_3, color='seagreen', marker='o') 
ax[2, 0].set_title('Original Sales', fontsize=24)
ax[2, 1].plot(y_a3, color='red', marker='.') 
ax[2, 1].set_title('After Wavelet Denoising', fontsize=24)

plt.show()

## Stores and States

### Rolling Average Price vs. Time for each store

In [ ]:

past_sales = sales_train_val.set_index('id')[req_col] \
    .T \
    .merge(calendar.set_index('d')['date'],
           left_index=True,
           right_index=True,
            validate='1:1') \
    .set_index('date')

In [ ]:
store_list = selling_prices['store_id'].unique()

means = []
fig = go.Figure()

for s in store_list:
    store_items = [c for c in past_sales.columns if s in c]
    data = past_sales[store_items].sum(axis=1).rolling(90).mean()
    means.append(np.mean(past_sales[store_items].sum(axis=1)))
    fig.add_trace(go.Scatter(x=np.arange(len(data)), y=data, name=s))
    
fig.update_layout(yaxis_title="Sales", xaxis_title="Time", title="Rolling Average Sales vs. Time (per store)")

The CA_3 has significantly more sales as compared to any other stores in US.

### Rolling Average Sales vs. Store name

In [ ]:
fig = go.Figure()

for i, s in enumerate(store_list):
        store_items = [c for c in past_sales.columns if s in c]
        data = past_sales[store_items].sum(axis=1).rolling(90).mean()
        fig.add_trace(go.Box(x=[s]*len(data), y=data, name=s))
    
fig.update_layout(yaxis_title="Sales", xaxis_title="Stores", title="Rolling Average Sales vs. Store name ")



### Mean sales vs. Store name

In [ ]:
df = pd.DataFrame(np.transpose([means, store_list]))
df.columns = ["Mean sales", "Store name"]
px.bar(df, y="Mean sales", x="Store name", color="Store name", title="Mean sales vs. Store name")

### Rolling Average Sales vs. Time (California)

In [ ]:
greens = ["mediumaquamarine", "mediumseagreen", "seagreen", "green"]
store_list = selling_prices['store_id'].unique()
fig = go.Figure()
means = []
stores = []
for i, s in enumerate(store_list):
    if "ca" in s or "CA" in s:
        store_items = [c for c in past_sales.columns if s in c]
        data = past_sales[store_items].sum(axis=1).rolling(90).mean()
        means.append(np.mean(past_sales[store_items].sum(axis=1)))
        stores.append(s)
        fig.add_trace(go.Scatter(x=np.arange(len(data)), y=data, name=s, marker=dict(color=greens[i])))
    
fig.update_layout(yaxis_title="Sales", xaxis_title="Time", title="Rolling Average Sales vs. Time (California)")


### Rolling Average Sales vs. Store name (California)

In [ ]:
fig = go.Figure()

for i, s in enumerate(store_list):
    if "ca" in s or "CA" in s:
        store_items = [c for c in past_sales.columns if s in c]
        data = past_sales[store_items].sum(axis=1).rolling(90).mean()
        fig.add_trace(go.Box(x=[s]*len(data), y=data, name=s, marker=dict(color=greens[i])))
    
fig.update_layout(yaxis_title="Sales", xaxis_title="Time", title="Rolling Average Sales vs. Store name (California)")

### Mean sales vs. Store name (California)

In [ ]:
df = pd.DataFrame(np.transpose([means, stores]))
df.columns = ["Mean sales", "Store name"]
px.bar(df, y="Mean sales", x="Store name", color="Store name", title="Mean sales vs. Store name", color_continuous_scale=greens)


fig = go.Figure(data=[
    go.Bar(name='', x=stores, y=means, marker={'color' : greens})])

fig.update_layout(title="Mean sales vs. Store name (California)", yaxis=dict(title="Mean sales"), xaxis=dict(title="Store name"))
fig.update_layout(barmode='group')
fig.show()


### Rolling Average Price vs. Time (WI)

In [ ]:
purples = ["thistle", "violet", "purple", "indigo"]
store_list = selling_prices['store_id'].unique()
fig = go.Figure()
means = []
stores = []
for i, s in enumerate(store_list):
    if "wi" in s or "WI" in s:
        store_items = [c for c in past_sales.columns if s in c]
        data = past_sales[store_items].sum(axis=1).rolling(90).mean()
        means.append(np.mean(past_sales[store_items].sum(axis=1)))
        stores.append(s)
        fig.add_trace(go.Scatter(x=np.arange(len(data)), y=data, name=s, marker=dict(color=purples[i%len(purples)])))
    
fig.update_layout(yaxis_title="Sales", xaxis_title="Time", title="Rolling Average Sales vs. Time (Wisconsin)")

In [ ]:
fig = go.Figure()

for i, s in enumerate(store_list):
    if "wi" in s or "WI" in s:
        store_items = [c for c in past_sales.columns if s in c]
        data = past_sales[store_items].sum(axis=1).rolling(90).mean()
        fig.add_trace(go.Box(x=[s]*len(data), y=data, name=s, marker=dict(color=purples[i%len(purples)])))
    
fig.update_layout(yaxis_title="Sales", xaxis_title="Time", title="Rolling Average Sales vs. Store name (Wisconsin)")

# Feature Engineering

### Memory Reduction

In [ ]:
def reduce_mem_usage(df, verbose=True):
    
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)  
                    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

#### Calendar data

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

def prep_calendar(df):
    df = df.drop(["date", "weekday"], axis=1)
    df = df.assign(d = df.d.str[2:].astype(int)) # Convert d column values from d_2 to 2
    df = df.fillna("missing")
    cols = list(set(df.columns) - {"wm_yr_wk", "d"})
    df[cols] = OrdinalEncoder(dtype="int").fit_transform(df[cols])
    df = reduce_mem_usage(df)
    return df

calendar = prep_calendar(calendar)

#### Selling Prices data

In [ ]:
def prep_selling_prices(df):
    gr = df.groupby(["store_id", "item_id"])["sell_price"]
    df["sell_price_rel_diff"] = gr.pct_change()
    df["sell_price_roll_sd7"] = gr.transform(lambda x: x.rolling(7).std())
    df["sell_price_cumrel"] = (gr.shift(0) - gr.cummin()) / (1 + gr.cummax() - gr.cummin())
    df = reduce_mem_usage(df)
    return df

selling_prices = prep_selling_prices(selling_prices)

#### Sales data

Reshaping

We now reshape the data from wide to long, using "id" as fixed and swapping "d_x" columns. Along this process, we also add structure for submission data and reduce data size.

In [ ]:
def reshape_sales(df, drop_d = None):
    if drop_d is not None:
        df = df.drop(["d_" + str(i + 1) for i in range(drop_d)], axis=1)
    df = df.assign(id=df.id.str.replace("_validation", ""))
    df = df.reindex(columns=df.columns.tolist() + ["d_" + str(1913 + i + 1) for i in range(2 * 28)]) # Validation data + Testing data
    df = df.melt(id_vars=["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"],
                 var_name='d', value_name='demand') 
    # It will unpivot each row data of sales, new rows will be created for each of the days 
    # (i.e 1912 rows for each product id) and quantity will be showed in 'demand' column.
    
    df = df.assign(d=df.d.str[2:].astype("int16"))
    return df

In [ ]:
%%time
sales = reshape_sales(sales_train_val, 1000)

Distribution of response

In [ ]:
sns.countplot(sales["demand"][sales["demand"] <= 10]);

Add Time lagged features

In [ ]:
def prep_sales(df):
    
    # shift - Shifts the data downwards by 28 values.
    # rolling - Provides rolling window calculations over the underlying data
    df['lag_t28'] = df.groupby(['id'])['demand'].transform(lambda x: x.shift(28))
    df['rolling_mean_t7'] = df.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).mean())
    df['rolling_mean_t30'] = df.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).mean())
    df['rolling_mean_t60'] = df.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(60).mean())
    df['rolling_mean_t90'] = df.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(90).mean())
    df['rolling_mean_t180'] = df.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(180).mean())
    df['rolling_std_t7'] = df.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).std())
    df['rolling_std_t30'] = df.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).std())

    # Remove rows with NAs except for submission rows. rolling_mean_t180 was selected as it produces most missings
    df = df[(df.d >= 1914) | (pd.notna(df.rolling_mean_t180))]
    df = reduce_mem_usage(df)

    return df



In [ ]:
%%time
sales = prep_sales(sales)

### Combine data sources

In [ ]:
sales = sales.merge(calendar, how="left", on="d")
gc.collect()

In [ ]:
print(sales.shape)
sales.head()

In [ ]:
sales = sales.merge(selling_prices, how="left", on=["wm_yr_wk", "store_id", "item_id"])
sales.drop(["wm_yr_wk"], axis=1, inplace=True)
gc.collect()

In [ ]:
print(sales.shape)
sales.head()

## Prepare data for Modelling

### Ordinal Encoding of remaing categories

In [ ]:
cat_id_cols = ["item_id", "dept_id", "store_id", "cat_id", "state_id"]
cat_cols = cat_id_cols + ["wday", "month", "year", "event_name_1", 
                          "event_type_1", "event_name_2", "event_type_2"]

# In loop to minimize memory use
for i, v in tqdm(enumerate(cat_id_cols)):
    sales[v] = OrdinalEncoder(dtype="int").fit_transform(sales[[v]])

sales = reduce_mem_usage(sales)
gc.collect()

### Impute Numeric Columns

In [ ]:
num_cols = ["sell_price", "sell_price_rel_diff", "sell_price_roll_sd7", "sell_price_cumrel",
            "lag_t28", "rolling_mean_t7", "rolling_mean_t30", "rolling_mean_t60", 
            "rolling_mean_t90", "rolling_mean_t180", "rolling_std_t7", "rolling_std_t30"]

bool_cols = ["snap_CA", "snap_TX", "snap_WI"]

dense_cols = num_cols + bool_cols

# Need to do column by column due to memory constraints
for i, v in tqdm(enumerate(num_cols)):
    sales[v] = sales[v].fillna(sales[v].median())
    


In [ ]:
print(sales.shape)
sales.head()

### Separate submission data and reconstruct id columns

In [ ]:
test = sales[sales.d >= 1914]

test = test.assign(id=test.id + "_" + np.where(test.d <= 1941, "validation", "evaluation"),
                   F="F" + (test.d - 1913 - 28 * (test.d > 1941)).astype("str"))
# test.head()
gc.collect()

In [ ]:
print(test.shape)
test.head()

### Make Training data

In [ ]:
# Input dict for training with a dense array and separate inputs for each embedding input
def make_X(df):
    # First item of dictionary will be values of all the columns in dense_cols
    X = {"dense1": df[dense_cols].to_numpy()}
    
    # Then, insert other columns in dictionary.
    for i, v in enumerate(cat_cols):
        X[v] = df[[v]].to_numpy()
    
    return X

In [ ]:
# Submission data
X_test = make_X(test)

In [ ]:
#X_test

In [ ]:
# One month of validation data
flag = (sales.d < 1914) & (sales.d >= 1914 - 28)

valid = (make_X(sales[flag]),sales["demand"][flag])

In [ ]:
#flag

In [ ]:
# Rest is used for training
flag = sales.d < 1914 - 28

X_train = make_X(sales[flag])
y_train = sales["demand"][flag]

In [ ]:
print(y_train.shape)
print(len(X_train['state_id']))

In [ ]:
# del sales, flag
gc.collect()

In [ ]:
np.unique(X_train["state_id"])

# The Model

## Architecture with Encoding

In [ ]:
def create_model(lr=0.002):
    tf.random.set_seed(173)

    tf.keras.backend.clear_session()
    gc.collect()

    # Dense input
    dense_input = Input(shape=(len(dense_cols), ), name='dense1')

    # Embedding input
    wday_input = Input(shape=(1,), name='wday')
    month_input = Input(shape=(1,), name='month')
    year_input = Input(shape=(1,), name='year')
    event_name_1_input = Input(shape=(1,), name='event_name_1')
    event_type_1_input = Input(shape=(1,), name='event_type_1')
    event_name_2_input = Input(shape=(1,), name='event_name_2')
    event_type_2_input = Input(shape=(1,), name='event_type_2')
    item_id_input = Input(shape=(1,), name='item_id')
    dept_id_input = Input(shape=(1,), name='dept_id')
    store_id_input = Input(shape=(1,), name='store_id')
    cat_id_input = Input(shape=(1,), name='cat_id')
    state_id_input = Input(shape=(1,), name='state_id')

    wday_emb = Flatten()(Embedding(7, 1)(wday_input))
    month_emb = Flatten()(Embedding(12, 1)(month_input))
    year_emb = Flatten()(Embedding(6, 1)(year_input))
    event_name_1_emb = Flatten()(Embedding(31, 1)(event_name_1_input))
    event_type_1_emb = Flatten()(Embedding(5, 1)(event_type_1_input))
    event_name_2_emb = Flatten()(Embedding(5, 1)(event_name_2_input))
    event_type_2_emb = Flatten()(Embedding(5, 1)(event_type_2_input))

    item_id_emb = Flatten()(Embedding(3049, 3)(item_id_input))
    dept_id_emb = Flatten()(Embedding(7, 1)(dept_id_input))
    store_id_emb = Flatten()(Embedding(10, 1)(store_id_input))
    cat_id_emb = Flatten()(Embedding(3, 1)(cat_id_input))
    state_id_emb = Flatten()(Embedding(3, 1)(state_id_input))

    # Combine dense and embedding parts and add dense layers. Exit on linear scale.
    x = concatenate([dense_input, wday_emb, month_emb, year_emb, 
                     event_name_1_emb, event_type_1_emb, 
                     event_name_2_emb, event_type_2_emb, 
                     item_id_emb, dept_id_emb, store_id_emb,
                     cat_id_emb, state_id_emb])
    
    x = Dense(150, activation="tanh")(x)
    x = Dense(75, activation="tanh")(x)
    x = Dense(10, activation="tanh")(x)
    
    outputs = Dense(1, activation="linear", name='output')(x)

    inputs = {"dense1": dense_input, "wday": wday_input, "month": month_input, "year": year_input, 
              "event_name_1": event_name_1_input, "event_type_1": event_type_1_input,
              "event_name_2": event_name_2_input, "event_type_2": event_type_2_input,
              "item_id": item_id_input, "dept_id": dept_id_input, "store_id": store_id_input, 
              "cat_id": cat_id_input, "state_id": state_id_input}

    # Connect input and output
    model = Model(inputs, outputs)

    model.compile(loss=keras.losses.mean_squared_error,
                  metrics=["mse",'accuracy'],
                  optimizer=keras.optimizers.RMSprop(learning_rate=lr))
    return model

In [ ]:
model = create_model(0.0002)
model.summary()
keras.utils.plot_model(model, 'model.png', show_shapes=True)

## Calculate derivatives and fit model

In [ ]:
%%time
history = model.fit(X_train, 
                    y_train,
                    batch_size=10000,
                    epochs=30,
                    shuffle=True,
                    validation_data=valid)

## Plot the evaluation metrics over epochs

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()

In [ ]:
history.history['accuracy']

In [ ]:
#history.history["val_loss"]

In [ ]:
model.save('model.h5')

In [79]:
pred = model.predict(X_test, batch_size=10000)

In [80]:
pred

array([[0.86706734],
       [0.24321413],
       [0.4167776 ],
       ...,
       [0.79215336],
       [0.83024526],
       [1.6213727 ]], dtype=float32)

In [81]:
test["demand"] = pred.clip(0)

submission = test.pivot(index="id", columns="F", values="demand").reset_index()[sample_submission.columns]
submission = sample_submission[["id"]].merge(submission, how="left", on="id")
submission.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.867067,0.731379,0.695733,0.716980,0.769611,1.106603,1.232728,0.671543,0.800339,0.838110,1.020043,0.976156,1.341891,1.412790,0.939812,0.830382,0.731211,0.769355,0.919425,1.254992,1.048525,0.856481,0.792503,0.801074,0.961350,0.920485,1.153962,1.198044
1,HOBBIES_1_002_CA_1_validation,0.243214,0.240822,0.236869,0.252791,0.281660,0.450310,0.505220,0.283257,0.278984,0.261418,0.275639,0.279524,0.363667,0.324843,0.247043,0.233832,0.198698,0.204869,0.243810,0.253767,0.266525,0.187845,0.170848,0.158507,0.190408,0.214077,0.276338,0.263894
2,HOBBIES_1_003_CA_1_validation,0.416778,0.386748,0.383130,0.404125,0.446214,0.771846,0.832242,0.368726,0.371378,0.324697,0.378424,0.380965,0.466274,0.458519,0.348081,0.291299,0.309996,0.326883,0.384203,0.513569,0.547988,0.422734,0.410387,0.418079,0.420147,0.489188,0.671137,0.681387
3,HOBBIES_1_004_CA_1_validation,1.783302,1.585225,1.535234,1.544305,1.782946,2.555187,2.785339,1.562469,1.544987,1.550182,1.600919,1.720318,2.403758,2.564982,1.748175,1.617731,1.551488,1.600005,1.866258,1.968695,2.181638,1.737844,1.628412,1.507592,1.614290,1.815387,2.410531,2.407449
4,HOBBIES_1_005_CA_1_validation,1.106168,1.028432,1.011627,1.098235,1.229120,2.015413,2.343323,1.141635,1.108929,1.111515,1.227440,1.238673,1.705707,1.684716,1.184083,1.121546,1.040460,1.092880,1.285316,1.514632,1.512653,1.138271,1.065721,1.048442,1.100108,1.240178,1.645002,1.732883


In [82]:
submission[sample_submission.id=="FOODS_1_001_TX_2_validation"].head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
16857,FOODS_1_001_TX_2_validation,0.510383,0.47488,0.460658,0.492498,0.535676,0.706077,0.728011,0.445011,0.460722,0.392192,0.456068,0.500787,0.532115,0.578242,0.457259,0.383325,0.439048,0.483381,0.515842,0.571743,0.63663,0.447298,0.423521,0.409238,0.420473,0.455519,0.554804,0.535417


In [83]:
submission.to_csv("submission.csv", index=False)